In [107]:
import pandas as pd
import numpy as np

In [66]:
import os

print("Current working directory:", os.getcwd())

# Check if the file exists
print("Does the file exist?", os.path.exists('Input/subroutes_completed.csv'))

Current working directory: /Users/keyifan/Library/Mobile Documents/com~apple~CloudDocs/UofT/24Winter/499datacleaning
Does the file exist? True


In [101]:
#Try to merge from 1995 to 2021??

# Load the datasets (replace 'path_to...' with the actual file paths)
data_2000_2009 = pd.read_csv('Input/cleaned_BOD/(BOD5)_PWQMN_2000_2009.csv')
data_1990_1999 = pd.read_csv('Input/cleaned_BOD/(BOD5)_PWQMN_1990_1999.csv')
data_2010_2018 = pd.read_csv('Input/cleaned_BOD/(BOD5)_PWQMN_2010_2018.csv')
data_2019_2021 = pd.read_csv('Input/cleaned_BOD/(BOD5)_PWQMN_2019_2021.csv')

In [102]:
# Convert the DATE column to datetime and then to YYYYMMDD format
data_1990_1999['DATE'] = pd.to_datetime(data_1990_1999['DATE']).dt.strftime('%Y%m%d')
# Rename columns in the 1990-1999 dataset as they dont match the other datasets
data_1990_1999.rename(columns={
    'PARAMETER CODE': 'PARM',
    'PARAMETER DESCRIPTION': 'PARM_DESCRIPTION',
    'DATE': 'DATE_YYYYMMDD',
    'TIME': 'TIME_HH:MM',
    'FIELD NUMBER': 'FIELD_NO',
    'RESULT VALUE': 'RESULT',
    'VALUE QUALIFIER': 'VALUQUALIFI',
    'REMARK CODE': 'REMARK_CODE',  # Assuming REMARK CODE1 is the correct one to keep
    'ANALYTICAL METHOD': 'ANALMETHOD'
}, inplace=True)
data_2019_2021.drop(columns=['_id'], inplace=True)  # Drop the _id column
data_1990_1999.drop(columns=['REMARK CODE1'], inplace=True)


# Drop the YEAR column from other datasets
data_2000_2009.drop(columns=['YEAR'], inplace=True)
data_2010_2018.drop(columns=['YEAR'], inplace=True)
data_2019_2021.drop(columns=['YEAR'], inplace=True)


In [109]:


# Get the column order from one of the datasets (e.g., 2000-2009)
column_order = data_2000_2009.columns

# Reorder the 1990-1999 dataset columns to match the order
data_1990_1999 = data_1990_1999[column_order]

# Now check if the columns in all datasets match in order
if (np.array_equal(data_1990_1999.columns, column_order) and
    np.array_equal(data_2000_2009.columns, column_order) and
    np.array_equal(data_2010_2018.columns, column_order) and
    np.array_equal(data_2019_2021.columns, column_order)):
    print("Column orders match. Proceeding with merge.")
    combined_data = pd.concat([data_1990_1999, data_2000_2009, data_2010_2018, data_2019_2021], ignore_index=True)
else:
    print("Column orders do not match. Please check the column orders.")

Column orders match. Proceeding with merge.


In [111]:

# Concatenate the datasets
combined_data = pd.concat([data_1990_1999, data_2000_2009, data_2010_2018, data_2019_2021], ignore_index=True)

combined_data.to_csv('Output/pollution_combined.csv', index=False)

print("Datasets merged successfully and saved.")

Datasets merged successfully and saved.


In [112]:
subroute_data = pd.read_csv('Input/subroutes_completed.csv')  # forward slashes
pollution_data = combined_data
stations_data = pd.read_csv('Input/stations_completed.csv')


In [113]:
#side step to reformat the station id in stations data 
#by removing the decimal and any digits following it
# Load the data

# Convert STATION to integer
stations_data['STATION'] = stations_data['STATION'].astype(int)

In [114]:
#drop any row of subroutes_data if the column downstream_station_id is null
subroute_data = subroute_data.dropna(subset=['downstream_station_id'])

In [116]:

# For convenience, add a prefix to the pollution data columns
pollution_data = pollution_data.add_prefix('pollution_')

# Merge subroute data with pollution data for upstream
merged_upstream = pd.merge(subroute_data, pollution_data, how='left', left_on='upstream_station_id', right_on='pollution_STATION')

# Merge subroute data with pollution data for downstream
merged_downstream = pd.merge(subroute_data, pollution_data, how='left', left_on='downstream_station_id', right_on='pollution_STATION')

# Combine the upstream and downstream data on the same dates
merged_data = pd.merge(merged_upstream, merged_downstream, how='inner', left_on=['subroute_id', 'pollution_DATE_YYYYMMDD'], right_on=['subroute_id', 'pollution_DATE_YYYYMMDD'], suffixes=('_upstream', '_downstream'))

# Filter out rows where pollution data is missing for either upstream or downstream
merged_data = merged_data.dropna(subset=['pollution_RESULT_upstream', 'pollution_RESULT_downstream'])

# Check the first few rows of the merged data
print(merged_data.head())

   subroute_id  upstream_station_id_upstream  downstream_station_id_upstream  \
0            1                    3007703502                    3.007704e+09   
1            1                    3007703502                    3.007704e+09   
2            1                    3007703502                    3.007704e+09   
3            4                    3007702202                    3.007702e+09   
4            4                    3007702202                    3.007702e+09   

   greenbelt_length_upstream  total_length_upstream  \
0               31989.069498           31989.069498   
1               31989.069498           31989.069498   
2               31989.069498           31989.069498   
3                   0.000000           29519.738894   
4                   0.000000           29519.738894   

   pollution_STATION_upstream pollution_PARM_upstream  \
0                3.007704e+09                    BOD5   
1                3.007704e+09                    BOD5   
2                

In [117]:
merged_data.to_csv('Output/merge2.csv', index=False)

In [120]:
# Dropping redundant columns
columns_to_drop = [
    'pollution_STATION_upstream', 'pollution_PARM_upstream', 
    'pollution_PARM_DESCRIPTION_upstream',
    'pollution_TIME_HH:MM_upstream', 'pollution_FIELD_NO_upstream', 
    'pollution_REMARK_CODE_upstream', 'pollution_VALUQUALIFI_upstream', 
    'upstream_station_id_downstream', 'downstream_station_id_downstream',
    'greenbelt_length_downstream', 'total_length_downstream',
    'pollution_STATION_downstream', 'pollution_PARM_downstream',
    'pollution_PARM_DESCRIPTION_downstream', 
    'pollution_TIME_HH:MM_downstream', 'pollution_FIELD_NO_downstream',
    'pollution_REMARK_CODE_downstream', 'pollution_VALUQUALIFI_downstream',
]

merged_data = merged_data.drop(columns=columns_to_drop)

# Renaming columns for clarity
merged_data = merged_data.rename(columns = {
    'greenbelt_length_upstream': 'gb_length',
    'total_length_upstream': 'length',
    'pollution_RESULT_upstream': 'Upstream_RESULT',
    'pollution_RESULT_downstream': 'Downstream_RESULT',
    'pollution_UNITS_upstream': 'UNITS',  # Assuming units are same for upstream and downstream
    'pollution_ANALMETHOD_upstream': 'ANALMETHOD',
    'upstream_station_id_upstream': 'upstream_station_id',
    'downstream_station_id_upstream': 'downstream_station_id'
})

#calculate a new column gb_share = gb_length/length
merged_data['gb_share'] = merged_data['gb_length'] / merged_data['length']

merged_data['downstream_station_id'] = merged_data['downstream_station_id'].astype(int)

# Display the revised DataFrame
print(merged_data.head())


merged_data.to_csv('Output/merge3.csv', index=False)


   subroute_id  upstream_station_id  downstream_station_id     gb_length  \
0            1           3007703502             3007704002  31989.069498   
1            1           3007703502             3007704002  31989.069498   
2            1           3007703502             3007704002  31989.069498   
3            4           3007702202             3007702302      0.000000   
4            4           3007702202             3007702302      0.000000   

         length pollution_DATE_YYYYMMDD  Upstream_RESULT                UNITS  \
0  31989.069498                20030512              1.5  MILLIGRAM PER LITER   
1  31989.069498                20040310              0.2  MILLIGRAM PER LITER   
2  31989.069498                20040419              1.0  MILLIGRAM PER LITER   
3  29519.738894                19900222              0.7  MILLIGRAM PER LITER   
4  29519.738894                19900322              0.1  MILLIGRAM PER LITER   

  ANALMETHOD  Downstream_RESULT pollution_UNITS_downstre

In [121]:

# Prepare stations data with gb_direction
stations_data['gb'] = stations_data['gb_direction'].apply(lambda x: 1 if x == 'Inside' else 0)


In [122]:
# Merge for upstream stations
merged_data2 = pd.merge(merged_data, stations_data[['STATION', 'gb', 'CSDUID', 'SessionName']], left_on='upstream_station_id', right_on='STATION', how='left')
merged_data2.rename(columns={'gb': 'us_gb', 'CSDUID': 'us_CSDUID', 'SessionName': 'us_SessionName'}, inplace=True)

# Merge for downstream stations (use merged_data2 from the previous step)
merged_data2 = pd.merge(merged_data2, stations_data[['STATION', 'gb', 'CSDUID', 'SessionName']], left_on='downstream_station_id', right_on='STATION', how='left', suffixes=('', '_downstream'))
merged_data2.rename(columns={'gb': 'ds_gb', 'CSDUID': 'ds_CSDUID', 'SessionName': 'ds_SessionName'}, inplace=True)

# Drop the STATION columns from the merges
merged_data2.drop(columns=['STATION', 'STATION_downstream'], inplace=True)


In [123]:
#reformat the YYYYMMDD
merged_data2['pollution_DATE_YYYYMMDD'] = merged_data2['pollution_DATE_YYYYMMDD'].astype(str)

# Extract year, month, and day
merged_data2['YYYY'] = merged_data2['pollution_DATE_YYYYMMDD'].str[:4]
merged_data2['MM'] = merged_data2['pollution_DATE_YYYYMMDD'].str[4:6]
merged_data2['DD'] = merged_data2['pollution_DATE_YYYYMMDD'].str[6:8]

# Convert the extracted columns to integers if necessary
merged_data2['YYYY'] = merged_data2['YYYY'].astype(int)
merged_data2['MM'] = merged_data2['MM'].astype(int)
merged_data2['DD'] = merged_data2['DD'].astype(int)

In [124]:
#write a column called 'post_gb'to the merged data if the date is after 28 feb of 2005, the value = 1 else 0
merged_data2['post_gb'] = merged_data2.apply(lambda x: 1 if (x['YYYY'] > 2005) or (x['YYYY'] == 2005 and x['MM'] > 2) else 0, axis=1)

In [125]:
# Display the first few rows of the final DataFrame
print(merged_data2.head())

# Save to CSV
merged_data2.to_csv('Output/merge5.csv', index=False)


   subroute_id  upstream_station_id  downstream_station_id     gb_length  \
0            1           3007703502             3007704002  31989.069498   
1            1           3007703502             3007704002  31989.069498   
2            1           3007703502             3007704002  31989.069498   
3            4           3007702202             3007702302      0.000000   
4            4           3007702202             3007702302      0.000000   

         length pollution_DATE_YYYYMMDD  Upstream_RESULT                UNITS  \
0  31989.069498                20030512              1.5  MILLIGRAM PER LITER   
1  31989.069498                20040310              0.2  MILLIGRAM PER LITER   
2  31989.069498                20040419              1.0  MILLIGRAM PER LITER   
3  29519.738894                19900222              0.7  MILLIGRAM PER LITER   
4  29519.738894                19900322              0.1  MILLIGRAM PER LITER   

  ANALMETHOD  Downstream_RESULT  ... us_gb  us_CSDUID  u

In [ ]:
#add control variables to the merged data